In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

import os

import warnings
warnings.filterwarnings('ignore')

from env import host, user, password
import acquire

# 1 <br>
Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
- Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.
- Only include properties that include a latitude and longitude value.

In [3]:
df = acquire.get_zillow_data()
df.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,id,logerror,transactiondate,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,...,None,None,None,None,0,0.025595,2017-01-01,Single Family Residential,None,None
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,...,None,None,None,None,1,0.055619,2017-01-01,Single Family Residential,None,None
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,...,None,None,None,None,2,0.005383,2017-01-01,Single Family Residential,None,None
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,...,None,None,None,Central,3,-0.103410,2017-01-01,Single Family Residential,None,None
4,10887214,NaN,NaN,266.0,2.0,NaN,NaN,1.0,1970746,NaN,...,Central,None,None,Central,4,0.006940,2017-01-01,Condominium,None,None


# 2 <br>
Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [5]:
df.shape

(9944, 69)

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
parcelid,9944.0,1.300296e+07,3.873631e+06,1.071194e+07,1.156709e+07,1.253446e+07,1.417620e+07,1.676885e+08
typeconstructiontypeid,44.0,6.000000e+00,0.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00
storytypeid,9.0,7.000000e+00,0.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00
propertylandusetypeid,9944.0,2.616866e+02,5.785147e+00,3.100000e+01,2.610000e+02,2.610000e+02,2.660000e+02,2.750000e+02
heatingorsystemtypeid,6404.0,4.047002e+00,3.695730e+00,1.000000e+00,2.000000e+00,2.000000e+00,7.000000e+00,2.400000e+01
buildingclasstypeid,3.0,4.000000e+00,0.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00
architecturalstyletypeid,42.0,7.619048e+00,3.177300e+00,3.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,2.100000e+01
airconditioningtypeid,3137.0,1.774625e+00,2.914254e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.300000e+01
id,9944.0,1.497462e+06,8.604439e+05,4.580000e+02,7.580392e+05,1.489863e+06,2.245646e+06,2.982136e+06
basementsqft,9.0,5.723333e+02,4.258122e+02,1.000000e+02,2.240000e+02,5.120000e+02,9.000000e+02,1.416000e+03


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9944 entries, 0 to 9906
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      9944 non-null   int64  
 1   typeconstructiontypeid        44 non-null     float64
 2   storytypeid                   9 non-null      float64
 3   propertylandusetypeid         9944 non-null   float64
 4   heatingorsystemtypeid         6404 non-null   float64
 5   buildingclasstypeid           3 non-null      float64
 6   architecturalstyletypeid      42 non-null     float64
 7   airconditioningtypeid         3137 non-null   float64
 8   id                            9944 non-null   int64  
 9   basementsqft                  9 non-null      float64
 10  bathroomcnt                   9944 non-null   float64
 11  bedroomcnt                    9944 non-null   float64
 12  buildingqualitytypeid         6489 non-null   float64
 13  cal

In [9]:
df.bathroomcnt.value_counts()

2.0     4071
3.0     2179
1.0     1758
2.5      813
4.0      429
1.5      175
3.5      120
5.0      117
4.5       90
0.0       66
6.0       52
5.5       29
8.0       16
7.0       13
6.5        9
9.0        4
11.0       1
10.0       1
18.0       1
Name: bathroomcnt, dtype: int64

In [10]:
df.bedroomcnt.value_counts()

3.0     3863
2.0     2484
4.0     2242
5.0      562
1.0      455
6.0      160
0.0       97
8.0       47
7.0       23
9.0        7
10.0       2
14.0       1
12.0       1
Name: bedroomcnt, dtype: int64

# 3 <br>
Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [12]:
acquire.nulls_by_col(df)

,num_rows_missing,percent_rows_missing
parcelid,0,0.000000
typeconstructiontypeid,9900,99.557522
storytypeid,9935,99.909493
propertylandusetypeid,0,0.000000
heatingorsystemtypeid,3540,35.599356
...,...,...
logerror,0,0.000000
transactiondate,0,0.000000
propertylandusedesc,0,0.000000
storydesc,9935,99.909493


# 1 <br>
Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [13]:
acquire.nulls_by_row(df)

,num_cols_missing,percent_cols_missing,index
0,24,34.78260869565217,4
1,25,36.231884057971016,3
2,26,37.68115942028986,13
3,27,39.130434782608695,45
4,28,40.57971014492754,66
5,29,42.028985507246375,634
6,30,43.47826086956522,458
7,31,44.927536231884055,1171
8,32,46.3768115942029,1642
9,33,47.82608695652174,1859


# Prepare <br>
# 1 <br>
Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [14]:
acquire.handle_missing_values(df, prop_required_columns=0.5, prop_required_row=0.75)

,parcelid,propertylandusetypeid,heatingorsystemtypeid,id,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,heatingorsystemdesc,id,logerror,transactiondate,propertylandusedesc
0,14297519,261.0,NaN,1727539,3.5,4.0,NaN,3.5,3100.0,3100.0,...,1023282.0,2016.0,537569.0,11013.72,6.059063e+13,None,0,0.025595,2017-01-01,Single Family Residential
1,17052889,261.0,NaN,1387261,1.0,2.0,NaN,1.0,1465.0,1465.0,...,464000.0,2016.0,376000.0,5672.48,6.111001e+13,None,1,0.055619,2017-01-01,Single Family Residential
2,14186244,261.0,NaN,11677,2.0,3.0,NaN,2.0,1243.0,1243.0,...,564778.0,2016.0,479489.0,6488.30,6.059022e+13,None,2,0.005383,2017-01-01,Single Family Residential
3,12177905,261.0,2.0,2288172,3.0,4.0,8.0,3.0,2376.0,2376.0,...,145143.0,2016.0,36225.0,1777.51,6.037300e+13,Central,3,-0.103410,2017-01-01,Single Family Residential
4,10887214,266.0,2.0,1970746,3.0,3.0,8.0,3.0,1312.0,1312.0,...,119407.0,2016.0,45726.0,1533.89,6.037124e+13,Central,4,0.006940,2017-01-01,Condominium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9892,17191625,266.0,NaN,343711,2.5,2.0,NaN,2.5,1463.0,1463.0,...,402506.0,2016.0,201253.0,4295.36,6.111006e+13,None,9900,0.010798,2017-02-15,Condominium
9891,10809856,261.0,2.0,672759,3.0,4.0,7.0,3.0,3267.0,3267.0,...,951841.0,2016.0,420768.0,11536.28,6.037138e+13,Central,9899,0.052829,2017-02-15,Single Family Residential
9890,12815475,261.0,2.0,933731,3.0,4.0,8.0,3.0,2306.0,2306.0,...,107443.0,2016.0,24208.0,1595.23,6.037408e+13,Central,9898,0.008809,2017-02-15,Single Family Residential
9896,14281159,261.0,NaN,2010661,1.0,3.0,NaN,1.0,1047.0,1047.0,...,210005.0,2016.0,149219.0,2857.92,6.059063e+13,None,9904,-0.047269,2017-02-15,Single Family Residential


In [15]:
df.propertylandusetypeid.value_counts()

261.0    6694
266.0    2473
246.0     303
269.0     232
248.0      94
247.0      84
265.0      34
263.0      12
260.0       6
267.0       5
275.0       4
31.0        3
Name: propertylandusetypeid, dtype: int64

In [16]:
df.propertylandusedesc.value_counts()

Single Family Residential                     6694
Condominium                                   2473
Duplex (2 Units, Any Combination)              303
Planned Unit Development                       232
Quadruplex (4 Units, Any Combination)           94
Triplex (3 Units, Any Combination)              84
Cluster Home                                    34
Mobile Home                                     12
Residential General                              6
Cooperative                                      5
Manufactured, Modular, Prefabricated Homes       4
Commercial/Office/Residential Mixed Used         3
Name: propertylandusedesc, dtype: int64

In [17]:
df.isna().sum()

parcelid                     0
typeconstructiontypeid    9900
storytypeid               9935
propertylandusetypeid        0
heatingorsystemtypeid     3540
                          ... 
logerror                     0
transactiondate              0
propertylandusedesc          0
storydesc                 9935
typeconstructiondesc      9900
Length: 69, dtype: int64

# 3 <br>
Decide how to handle the remaining missing values: <br>
- Fill with constant value.
- Impute with mean, median, mode.
- Drop row/column

In [18]:
# Dropping any property type that is not for a single family or residential
# Propertytypeids = 31, 246, 247, 248, 260, 267, 269, 275
# Keeping 261, 263, 264, 265, 266
index_names = df[ df['propertylandusetypeid'] == 267].index
index_names

Int64Index([1744, 2437, 3796, 7458, 9599], dtype='int64')

In [19]:
df.drop(index_names, inplace = True)

In [20]:
df.propertylandusetypeid.value_counts()

261.0    6694
266.0    2473
246.0     303
269.0     232
248.0      94
247.0      84
265.0      34
263.0      12
260.0       6
275.0       4
31.0        3
Name: propertylandusetypeid, dtype: int64

In [21]:
df.propertylandusedesc.value_counts()

Single Family Residential                     6694
Condominium                                   2473
Duplex (2 Units, Any Combination)              303
Planned Unit Development                       232
Quadruplex (4 Units, Any Combination)           94
Triplex (3 Units, Any Combination)              84
Cluster Home                                    34
Mobile Home                                     12
Residential General                              6
Manufactured, Modular, Prefabricated Homes       4
Commercial/Office/Residential Mixed Used         3
Name: propertylandusedesc, dtype: int64

In [22]:
df.isna().sum()

parcelid                     0
typeconstructiontypeid    9895
storytypeid               9930
propertylandusetypeid        0
heatingorsystemtypeid     3540
                          ... 
logerror                     0
transactiondate              0
propertylandusedesc          0
storydesc                 9930
typeconstructiondesc      9895
Length: 69, dtype: int64

In [23]:
df.typeconstructiontypeid.value_counts()

6.0    44
Name: typeconstructiontypeid, dtype: int64

In [24]:
df.typeconstructiondesc.value_counts()

Frame    44
Name: typeconstructiondesc, dtype: int64

In [25]:
df.storytypeid.value_counts()

7.0    9
Name: storytypeid, dtype: int64

In [26]:
df.storydesc.value_counts()

Basement    9
Name: storydesc, dtype: int64

In [27]:
df.heatingorsystemdesc.value_counts()

Central       4222
Floor/Wall    1930
Yes            118
Forced air      98
Solar           16
None            11
Radiant          3
Baseboard        1
Name: heatingorsystemdesc, dtype: int64

In [28]:
df.typeconstructiondesc.value_counts()

Frame    44
Name: typeconstructiondesc, dtype: int64

In [29]:
df.shape

(9939, 69)

In [30]:
acquire.nulls_by_row(df)

,num_cols_missing,percent_cols_missing,index
0,24,34.78260869565217,4
1,25,36.231884057971016,3
2,26,37.68115942028986,13
3,27,39.130434782608695,45
4,28,40.57971014492754,66
5,29,42.028985507246375,634
6,30,43.47826086956522,458
7,31,44.927536231884055,1171
8,32,46.3768115942029,1641
9,33,47.82608695652174,1855


In [31]:
nulls = pd.DataFrame(df.isna().sum())
nulls[nulls>0].value_counts()

9936.0    3
9895.0    3
9930.0    3
9460.0    2
6803.0    2
6797.0    2
3540.0    2
9205.0    2
68.0      2
9897.0    2
9883.0    1
9861.0    1
8979.0    1
9827.0    1
9783.0    1
9674.0    1
9903.0    1
9493.0    1
9932.0    1
9848.0    1
1.0       1
8699.0    1
8087.0    1
8.0       1
7737.0    1
6039.0    1
3455.0    1
3366.0    1
3341.0    1
1075.0    1
529.0     1
178.0     1
28.0      1
27.0      1
24.0      1
17.0      1
7971.0    1
dtype: int64

In [32]:
df.isna().sum()

parcelid                     0
typeconstructiontypeid    9895
storytypeid               9930
propertylandusetypeid        0
heatingorsystemtypeid     3540
                          ... 
logerror                     0
transactiondate              0
propertylandusedesc          0
storydesc                 9930
typeconstructiondesc      9895
Length: 69, dtype: int64

In [33]:
cols_to_remove = ['typeconstructiontypeid', 'storytypeid','storydesc','typeconstructiondesc']
df = acquire.remove_columns(df, cols_to_remove)

In [34]:
df.isna().sum()

parcelid                       0
propertylandusetypeid          0
heatingorsystemtypeid       3540
buildingclasstypeid         9936
architecturalstyletypeid    9897
                            ... 
heatingorsystemdesc         3540
id                             0
logerror                       0
transactiondate                0
propertylandusedesc            0
Length: 65, dtype: int64

In [35]:
cols_to_remove = ['heatingorsystemtypeid','architecturalstyletypeid']
df = acquire.remove_columns(df, cols_to_remove)

In [36]:
df.isna().sum()

parcelid                    0
propertylandusetypeid       0
buildingclasstypeid      9936
airconditioningtypeid    6803
id                          0
                         ... 
heatingorsystemdesc      3540
id                          0
logerror                    0
transactiondate             0
propertylandusedesc         0
Length: 63, dtype: int64

In [37]:
cols_to_remove = ['buildingclasstypeid']
df = acquire.remove_columns(df, cols_to_remove)

In [38]:
df.isna().sum()

parcelid                    0
propertylandusetypeid       0
airconditioningtypeid    6803
id                          0
basementsqft             9930
                         ... 
heatingorsystemdesc      3540
id                          0
logerror                    0
transactiondate             0
propertylandusedesc         0
Length: 62, dtype: int64

In [39]:
cols_to_remove = ['basementsqft']
df = acquire.remove_columns(df, cols_to_remove)

In [40]:
df.isna().sum()

parcelid                    0
propertylandusetypeid       0
airconditioningtypeid    6803
id                          0
bathroomcnt                 0
                         ... 
heatingorsystemdesc      3540
id                          0
logerror                    0
transactiondate             0
propertylandusedesc         0
Length: 61, dtype: int64

In [41]:
missing = df.isnull().sum()
print('-----')
print(missing[missing > 0])

-----
airconditioningtypeid           6803
buildingqualitytypeid           3455
calculatedbathnbr                 68
decktypeid                      9861
finishedfloor1squarefeet        9205
calculatedfinishedsquarefeet      24
finishedsquarefeet12             529
finishedsquarefeet13            9936
finishedsquarefeet15            9493
finishedsquarefeet50            9205
finishedsquarefeet6             9883
fireplacecnt                    8979
fullbathcnt                       68
garagecarcnt                    6797
garagetotalsqft                 6797
hashottuborspa                  9783
lotsizesquarefeet               1075
poolcnt                         7971
poolsizesum                     9848
pooltypeid10                    9895
pooltypeid2                     9827
pooltypeid7                     8087
propertyzoningdesc              3366
regionidcity                     178
regionidneighborhood            6039
regionidzip                        8
threequarterbathnbr             

In [42]:
df = acquire.handle_missing_values(df, prop_required_columns=0.5, prop_required_row=0.75)

In [43]:
missing = df.isnull().sum()
print('-----')
print(missing[missing > 0])

-----
buildingqualitytypeid           3440
calculatedbathnbr                 53
calculatedfinishedsquarefeet       9
finishedsquarefeet12             514
fullbathcnt                       53
lotsizesquarefeet               1070
propertyzoningdesc              3352
regionidcity                     178
regionidzip                        8
unitcnt                         3326
yearbuilt                         13
structuretaxvaluedollarcnt        17
taxamount                          1
censustractandblock               26
heatingorsystemdesc             3525
dtype: int64


In [44]:
df.heatingorsystemdesc.value_counts()

Central       4222
Floor/Wall    1930
Yes            118
Forced air      98
Solar           16
None            11
Radiant          3
Baseboard        1
Name: heatingorsystemdesc, dtype: int64

In [45]:
df.shape

(9924, 34)

In [48]:
imp = SimpleImputer(strategy="most_frequent")

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9924 entries, 0 to 9906
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      9924 non-null   int64  
 1   propertylandusetypeid         9924 non-null   float64
 2   id                            9924 non-null   int64  
 3   bathroomcnt                   9924 non-null   float64
 4   bedroomcnt                    9924 non-null   float64
 5   buildingqualitytypeid         6484 non-null   float64
 6   calculatedbathnbr             9871 non-null   float64
 7   calculatedfinishedsquarefeet  9915 non-null   float64
 8   finishedsquarefeet12          9410 non-null   float64
 9   fips                          9924 non-null   float64
 10  fullbathcnt                   9871 non-null   float64
 11  latitude                      9924 non-null   float64
 12  longitude                     9924 non-null   float64
 13  lot

In [50]:
imp.fit_transform(df)

array([[14297519, 261.0, 1727539, ..., 0.025594901758400002,
        '2017-01-01', 'Single Family Residential'],
       [17052889, 261.0, 1387261, ..., 0.0556190874015, '2017-01-01',
        'Single Family Residential'],
       [14186244, 261.0, 11677, ..., 0.0053828530468900005, '2017-01-01',
        'Single Family Residential'],
       ...,
       [12815475, 261.0, 933731, ..., 0.008809434751829999, '2017-02-15',
        'Single Family Residential'],
       [14281159, 261.0, 2010661, ..., -0.047269379667, '2017-02-15',
        'Single Family Residential'],
       [17176731, 261.0, 1699290, ..., -0.008628105671950001,
        '2017-02-15', 'Single Family Residential']], dtype=object)

In [51]:
missing = df.isnull().sum()
print('-----')
print(missing[missing > 0])

-----
buildingqualitytypeid           3440
calculatedbathnbr                 53
calculatedfinishedsquarefeet       9
finishedsquarefeet12             514
fullbathcnt                       53
lotsizesquarefeet               1070
propertyzoningdesc              3352
regionidcity                     178
regionidzip                        8
unitcnt                         3326
yearbuilt                         13
structuretaxvaluedollarcnt        17
taxamount                          1
censustractandblock               26
heatingorsystemdesc             3525
dtype: int64


In [52]:
df.info

<bound method DataFrame.info of       parcelid  propertylandusetypeid       id  bathroomcnt  bedroomcnt  \
0     14297519                  261.0  1727539          3.5         4.0   
1     17052889                  261.0  1387261          1.0         2.0   
2     14186244                  261.0    11677          2.0         3.0   
3     12177905                  261.0  2288172          3.0         4.0   
4     10887214                  266.0  1970746          3.0         3.0   
...        ...                    ...      ...          ...         ...   
9892  17191625                  266.0   343711          2.5         2.0   
9891  10809856                  261.0   672759          3.0         4.0   
9890  12815475                  261.0   933731          3.0         4.0   
9896  14281159                  261.0  2010661          1.0         3.0   
9906  17176731                  261.0  1699290          3.0         4.0   

      buildingqualitytypeid  calculatedbathnbr  calculatedfinisheds

In [53]:
df.shape

(9924, 34)